<a href="https://colab.research.google.com/github/mswang12/minDQN/blob/main/minDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Minimal Deep Q-Network
We'll be showing how to code a minimal Deep Q-Network to solve the CartPole environment.

## Step 1. Import libraries and setup the environment

In [1]:
!pip show tensorflow
!pip show gymnasium

Name: tensorflow
Version: 2.16.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\Mike\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: tensorflow-intel
Required-by: 
Name: gymnasium
Version: 0.29.1
Summary: A standard API for reinforcement learning and a diverse set of reference environments (formerly Gym).
Home-page: 
Author: 
Author-email: Farama Foundation <contact@farama.org>
License: MIT License
Location: C:\Users\Mike\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: cloudpickle, farama-notifications, numpy, typing-extensions
Required-by: 


In [18]:
import gymnasium as gym
import tensorflow as tf
import numpy as np
from tensorflow import keras

from collections import deque
import time
import random

RANDOM_SEED = 5
tf.random.set_seed(RANDOM_SEED)

env = gym.make('CartPole-v1')
env.reset(seed=RANDOM_SEED)
# env.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print("Action Space: {}".format(env.action_space))
print("State space: {}".format(env.observation_space))

Action Space: Discrete(2)
State space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


## Step 2. Define the network architecture

In [19]:
# An episode a full game
train_episodes = 100
test_episodes = 30

def agent(state_shape, action_shape):
    """ The agent maps X-states to Y-actions
    e.g. The neural network output is [.1, .7, .05, 0.05, .05, .05]
    The highest value 0.7 is the Q-Value.
    The index of the highest action (0.7) is action #1.
    """
    learning_rate = 0.001
    init = tf.keras.initializers.HeUniform()
    model = keras.Sequential()
    model.add(keras.layers.Dense(24, input_shape=state_shape, activation='relu', kernel_initializer=init))
    model.add(keras.layers.Dense(12, activation='relu', kernel_initializer=init))
    model.add(keras.layers.Dense(action_shape, activation='linear', kernel_initializer=init))
    model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model

def get_qs(model, state, step):
    return model.predict(state.reshape([1, state.shape[0]]))[0]


## Step 3. Define the train function using Experience Replay

In [20]:
def train(env, replay_memory, model, target_model, done):
    learning_rate = 0.7 # Learning rate
    discount_factor = 0.618

    MIN_REPLAY_SIZE = 1000
    if len(replay_memory) < MIN_REPLAY_SIZE:
        return

    batch_size = 64 * 2
    mini_batch = random.sample(replay_memory, batch_size)
    current_states = np.array([transition[0] for transition in mini_batch])
    current_qs_list = model.predict(current_states, verbose=0)
    new_current_states = np.array([transition[3] for transition in mini_batch])
    future_qs_list = target_model.predict(new_current_states, verbose=0)

    X = []
    Y = []
    for index, (observation, action, reward, new_observation, done) in enumerate(mini_batch):
        if not done:
            max_future_q = reward + discount_factor * np.max(future_qs_list[index])
        else:
            max_future_q = reward

        current_qs = current_qs_list[index]
        current_qs[action] = (1 - learning_rate) * current_qs[action] + learning_rate * max_future_q

        X.append(observation)
        Y.append(current_qs)
    model.fit(np.array(X), np.array(Y), batch_size=batch_size, verbose=0, shuffle=True)


## Step 4. Run the Deep Q-Network Algorithm

In [24]:
def main():
    epsilon = 0.5 # Epsilon-greedy algorithm in initialized at 1 meaning every step is random at the start
    max_epsilon = 1 # You can't explore more than 100% of the time
    min_epsilon = 0.01 # At a minimum, we'll always explore 1% of the time
    decay = 0.01

    # 1. Initialize the Target and Main models
    # Main Model (updated every 4 steps)
    print(env.observation_space.shape, env.action_space.n)
    model = agent(env.observation_space.shape, env.action_space.n)
    
    # model = keras.models.load_model("cartpole-dqn.keras")
    # Target Model (updated every 100 steps)
    target_model = agent(env.observation_space.shape, env.action_space.n)
    target_model.set_weights(model.get_weights())

    replay_memory = deque(maxlen=50_000)

    target_update_counter = 0

    # X = states, y = actions
    X = []
    y = []

    steps_to_update_target_model = 0

    for episode in range(train_episodes):
        total_training_rewards = 0
        observation, _ = env.reset()
        done = False
        while not done:
            steps_to_update_target_model += 1
            #if True:
            #    env.render()

            random_number = np.random.rand()
            # 2. Explore using the Epsilon Greedy Exploration Strategy
            if random_number <= epsilon:
                # Explore
                action = env.action_space.sample()
            else:
                # Exploit best known action
                # model dims are (batch, env.observation_space.n)
                encoded = observation
                # print(encoded)
                # print(encoded.shape)
                encoded_reshaped = encoded.reshape([1, encoded.shape[0]])
                # print(encoded_reshaped)
                # print(encoded_reshaped.shape)
                # print('--------------')
                predicted = model.predict(encoded_reshaped, verbose=0).flatten()
                action = np.argmax(predicted)
            new_observation, reward, done, trunc, info = env.step(action)
            replay_memory.append([observation, action, reward, new_observation, done])

            # 3. Update the Main Network using the Bellman Equation
            if steps_to_update_target_model % 4 == 0 or done:
                train(env, replay_memory, model, target_model, done)

            observation = new_observation
            total_training_rewards += reward

            if done:
                print('Total training rewards: {} after n steps = {} with final reward = {}'.format(total_training_rewards, episode, reward))
                total_training_rewards += 1

                if steps_to_update_target_model >= 100:
                    print('Copying main network weights to the target network weights')
                    target_model.set_weights(model.get_weights())
                    steps_to_update_target_model = 0
                break

        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay * episode)
    env.close()
    model.save('cartpole-dqn-m2.keras')
    target_model.save('cartpole-dqn-t2.keras')


if __name__ == '__main__':
    main()


(4,) 2
[ 0.01015702  0.04810306  0.03904772 -0.02577459]
(4,)
[[ 0.01015702  0.04810306  0.03904772 -0.02577459]]
(1, 4)
--------------
[ 0.01185005  0.4344663   0.04571516 -0.5262111 ]
(4,)
[[ 0.01185005  0.4344663   0.04571516 -0.5262111 ]]
(1, 4)
--------------
[ 0.02617651  0.23779224  0.03248317 -0.19871613]
(4,)
[[ 0.02617651  0.23779224  0.03248317 -0.19871613]]
(1, 4)
--------------
[0.03093236 0.04222111 0.02850885 0.10403418]
(4,)
[[0.03093236 0.04222111 0.02850885 0.10403418]]
(1, 4)
--------------
[ 0.03177678 -0.15329756  0.03058953  0.40557346]
(4,)
[[ 0.03177678 -0.15329756  0.03058953  0.40557346]]
(1, 4)
--------------


c:\Users\Mike\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[0.01864854 0.0400605  0.06140557 0.15192436]
(4,)
[[0.01864854 0.0400605  0.06140557 0.15192436]]
(1, 4)
--------------
[ 0.00929496 -0.5479094   0.08922283  1.0905432 ]
(4,)
[[ 0.00929496 -0.5479094   0.08922283  1.0905432 ]]
(1, 4)
--------------
[-0.00166323 -0.7440868   0.1110337   1.4098355 ]
(4,)
[[-0.00166323 -0.7440868   0.1110337   1.4098355 ]]
(1, 4)
--------------
[-0.01654496 -0.94039685  0.13923042  1.7350641 ]
(4,)
[[-0.01654496 -0.94039685  0.13923042  1.7350641 ]]
(1, 4)
--------------
Total training rewards: 21.0 after n steps = 0 with final reward = 1.0
Total training rewards: 14.0 after n steps = 1 with final reward = 1.0
Total training rewards: 16.0 after n steps = 2 with final reward = 1.0
Total training rewards: 25.0 after n steps = 3 with final reward = 1.0
[ 0.0014901  -0.2081284   0.03899319  0.32020643]
(4,)
[[ 0.0014901  -0.2081284   0.03899319  0.32020643]]
(1, 4)
--------------
Total training rewards: 16.0 after n steps = 4 with final reward = 1.0
[ 0.0377

KeyboardInterrupt: 

In [3]:
env = gym.make('CartPole-v1', render_mode='human')

model = keras.models.load_model("cartpole-dqn.keras")

observation, _ = env.reset()

appendedObservations = []
rewards = 0

for i in range(200):
  print(i)
  encoded = observation
  encoded_reshaped = encoded.reshape([1, encoded.shape[0]])
  predicted = model.predict(encoded_reshaped, verbose=0).flatten()
  action = np.argmax(predicted)
  new_observation, reward, terminated, truncated, info =env.step(action)
  appendedObservations.append(new_observation)
  observation = new_observation
  rewards += reward
  # time.sleep(0.1)
  if (terminated):
      break
print(rewards)
env.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171.0
